In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
import datetime

In [3]:
import requests
import json
header = {'Content-Type': 'application/json', 'Accept': 'application/json'}

### Авторизация доступа

In [4]:
# Выполняется запрос с неверным ключем
resp = requests.get("http://37.192.49.26:8085/train", headers= header, data=json.dumps({"key":1313}))             
print("Return Status: ", resp.status_code)
print(resp.text)

Return Status:  500
{"error": {"message": "Unauthorized access"}}


In [5]:
# Запрос с валидным ключем
resp = requests.get("http://37.192.49.26:8085/train", 
                    headers= header, 
                    data=json.dumps({"key":"671EFAC4B8D59302"}))             

print("Return Status: ",resp.status_code)
resp_dict = resp.json()
df_data_train = pd.DataFrame(resp_dict.get("data"))
df_data_train.head()

Return Status:  200


,ID,applicability,sentiment,text
0,0,1,1,I had the opportunity today to sample your ama...
1,1,1,1,If you are Razr owner...you must have this!
10,10,1,0,I was not happy with this item.
100,100,1,1,This is a great deal.
1000,1000,1,1,Bluetooth range is good - a few days ago I lef...


### Валидационный сэмпл

In [6]:
# Запрос с валидным ключем
#  можно сделать не более одного зароса в минуту
resp = requests.get("http://37.192.49.26:8085/valid_test", 
                    headers= header, 
                    data=json.dumps({"key":"671EFAC4B8D59302"}))             

print("Return Status: ",resp.status_code)
resp_dict = resp.json()
df_data_vlid = pd.DataFrame(resp_dict.get("data"))
df_data_vlid.head()

Return Status:  200


,ID,applicability,sentiment,text
0,1923,NaN,NaN,NaN
1,1924,NaN,NaN,NaN
10,1933,NaN,NaN,NaN
100,2023,NaN,NaN,NaN
101,2024,NaN,NaN,NaN


### Проверка результатов

In [7]:
# посмотрим как работает проверка результатов на валидационном сете
random.seed(2018)
df_data_vlid["sentiment"] = [random.randint(0, 1) for i in range(df_data_vlid.shape[0])]
df_data_vlid["applicability"] = [random.randint(0, 1) for i in range(df_data_vlid.shape[0])]
resp = requests.post("http://37.192.49.26:8085/valid_test", 
                    headers= header, 
                    data=json.dumps({"key":"671EFAC4B8D59302", "data":df_data_vlid.to_dict()}))             

print("Return Status: ",resp.status_code)
resp.json()

Return Status:  200


{'success': True,
 'scores': {'score': 0.2543331473751033,
  'f1_sent': 0.5154185022026431,
  'f1_appl': 0.49344978165938863}}

In [8]:
# посмотрим как работает проверка результатов на тренировочном сете
random.seed(2018)
df_data_train["sentiment"] = [random.randint(0, 1) for i in range(df_data_train.shape[0])]
df_data_train["applicability"] = [random.randint(0, 1) for i in range(df_data_train.shape[0])]
resp = requests.post("http://37.192.49.26:8085/train", 
                    headers= header, 
                    data=json.dumps({"key":"671EFAC4B8D59302", "data":df_data_train.to_dict()}))
print("Return Status: ",resp.status_code)
resp.json()

Return Status:  200


{'success': True,
 'scores': {'score': None, 'f1_sent': 0.5002552322613579, 'f1_appl': None}}

### Тестовые данные

In [9]:
# Запрос с валидным ключем
#  можно сделать не более одного зароса в минуту
resp = requests.get("http://37.192.49.26:8085/test", 
                    headers= header, 
                    data=json.dumps({"key":"671EFAC4B8D59302"}))             

print("Return Status: ",resp.status_code)
resp_dict = resp.json()
df_data_test = pd.DataFrame(resp_dict.get("data"))
df_data_test.head()

Return Status:  200


""


In [10]:
# Отрпавляем готовые резульаты на проверку
random.seed(2018)
df_data_test["sentiment"] = [random.randint(0, 1) for i in range(df_data_test.shape[0])]
df_data_test["applicability"] = [random.randint(0, 1) for i in range(df_data_test.shape[0])]
resp = requests.post("http://37.192.49.26:8085/test", 
                    headers= header, 
                    data=json.dumps({"key":"671EFAC4B8D59302", "data":df_data_test.to_dict()}))             

print("Return Status: ",resp.status_code)
resp_dict = resp.json()
print(resp_dict)

Return Status:  200
{'success': True, 'message': 'Submission will be opened 2019-03-03 17:00:00, now 2019-03-01 17:04:17.294795'}
